In [1]:
import tensorflow as tf
from tensorflow import keras

import mlflow
import mlflow.keras

print('TensorFlow version: {}'.format(tf.__version__))

TensorFlow version: 2.9.1


In [12]:
mlflow.set_tracking_uri("https://mlflow.sungin.ml:443")
mlflow.set_experiment("tf-fashion-mnist-experiment")

<Experiment: artifact_location='mlflow-artifacts:/2', experiment_id='2', lifecycle_stage='active', name='tf-fashion-mnist-experiment', tags={}>

In [13]:
## MLFlow 서버에 학습 결과 및 모델 저장

mlflow.tensorflow.autolog()
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_acc))


train_images.shape: (60000, 28, 28, 1), of float64
test_images.shape: (10000, 28, 28, 1), of float64
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 13, 13, 8)         80        
                                                                 
 flatten (Flatten)           (None, 1352)              0         
                                                                 
 Dense (Dense)               (None, 10)                13530     
                                                                 
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________


2022-08-23 16:31:56.292128: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022/08/23 16:31:57 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8944eaa3ec5d472faa7849719d8cd99d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.5389 - sparse_categorical_accuracy: 0.8079
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4000 - sparse_categorical_accuracy: 0.8586
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3643 - sparse_categorical_accuracy: 0.8696
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3447 - sparse_categorical_accuracy: 0.8773
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3304 - sparse_categorical_accuracy: 0.8805


INFO:tensorflow:Assets written to: /var/folders/t5/j16x5yxj6yb2f6tzgwvt3jk40000gn/T/tmpkomwtt1h/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/t5/j16x5yxj6yb2f6tzgwvt3jk40000gn/T/tmpkomwtt1h/model/data/model/assets
2022/08/23 16:32:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mzc01-lsi/anaconda3/envs/qraft-mlflow-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


313/313 [==============================] - 1s 2ms/step - loss: 0.3769 - sparse_categorical_accuracy: 0.8596
Test loss: 0.37686705589294434
Test accuracy: 0.8596000075340271


In [5]:
## BentoML에서 S3에 저장된 MLFlow 모델 로드 시 에러가 발생하여 추가로 로컬에도 모델을 저장함

import bentoml

bentoml.keras.save_model(
    'tf_fashion_mnist_model',
    model,
    signatures={
        'predict': {'batchable': True, 'batch_dim': 0},
    },
)

INFO:tensorflow:Assets written to: /var/folders/t5/j16x5yxj6yb2f6tzgwvt3jk40000gn/T/tmpuycg8q9cbentoml_model_tf_fashion_mnist_model/assets


INFO:tensorflow:Assets written to: /var/folders/t5/j16x5yxj6yb2f6tzgwvt3jk40000gn/T/tmpuycg8q9cbentoml_model_tf_fashion_mnist_model/assets


Model(tag="tf_fashion_mnist_model:744hbprb62fuxlg6", path="/Users/mzc01-lsi/bentoml/models/tf_fashion_mnist_model/744hbprb62fuxlg6/")

In [6]:
runner = bentoml.keras.get('tf_fashion_mnist_model').to_runner()
runner.init_local()

In [7]:
test = test_images[:2]
test.shape

(2, 28, 28, 1)

In [8]:
runner.predict.run(test)

1/1 [==============================] - 0s 86ms/step


array([[ -5.7994285, -10.12332  ,  -5.468986 ,  -6.1174626,  -5.774911 ,
          1.3063929,  -3.2462103,   2.0866206,  -0.2609799,   5.701401 ],
       [ -3.015513 , -18.553076 ,   4.0805845,  -8.353826 ,  -2.7561944,
        -28.514433 ,  -1.2832967, -35.27537  ,  -8.404896 , -23.74344  ]],
      dtype=float32)